In [ ]:
import pandas as pd

def clean_str(df: pd.DataFrame, columns: list[str]) -> pd.DataFrame:
    for col in columns:
        if col in df.columns:
            df[col] = df[col].fillna("").astype(str).str.strip().str.upper()
    return df

def clean_nan(value):
    return None if pd.isna(value) else value

def values_differ(current, value) -> bool:
    """Compara valores considerando fechas y None de forma robusta."""
    # Si ambos son None → no difieren
    if current is None and value is None:
        return False

    # Si uno es None y el otro no → difieren
    if (current is None) != (value is None):
        return True

    # Comparación especial para fechas
    if isinstance(current, (datetime, date)) and isinstance(value, (datetime, date)):
        curr_date = current.date() if isinstance(current, datetime) else current
        val_date = value.date() if isinstance(value, datetime) else value
        return curr_date != val_date

    # Comparación genérica
    return current != value

def preproccess_traffic(df: pd.DataFrame) -> pd.DataFrame:
    df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")
    df.drop_duplicates(subset=df.columns, inplace=True)
    df.dropna(subset=["file", "proveedor", "pasajero"], inplace=True)

    df = clean_str(df, ["estado", "moneda", "proveedor", "pasajero", "codigo_iata"])
    df.loc[:, "monto_a_pagar"] = round(df["monto_a_pagar"], 2)
    df.loc[:, "fecha_de_pago_proveedor"] =  pd.to_datetime(df["fecha_de_pago_proveedor"])
    df.loc[:, "fecha_servicio"] = pd.to_datetime(df["fecha_servicio"])
    df.loc[:, "fecha_out"] = pd.to_datetime(df["fecha_out"])
    df.loc[:, "fec_sal"] = pd.to_datetime(df["fecha_out"])
    
    text_cols = df.select_dtypes(include="object").columns
    df[text_cols] = df[text_cols].replace({"": None})
    return df

In [33]:
df = pd.read_excel(r"C:\Users\jsaldano\Documents\Procesar\excel.xlsx").drop("Unnamed: 0", axis=1)

In [37]:
df.loc[df.file == "CR0003"]

,id_orden,file,estado,moneda,fecha_servicio,fecha_out,pasajero,monto_a_pagar,proveedor,codigo_iata,fec_sal,fecha_de_pago_proveedor
237,251,CR0003,CX,D,2025-10-21,2025-10-21,AUTOM DE PAYMENT DET_RVA,567.0,LATITUD USHUAIA OPERADOR RECEPTIVO,USH,2025-10-27T00:00:00.000,2025-10-20
317,339,CR0003,CX,D,2025-10-21,2025-10-21,AUTOM DE PAYMENT DET_RVA,567.0,LATITUD USHUAIA OPERADOR RECEPTIVO,USH,2025-10-27T00:00:00.000,2025-10-20


In [17]:
df = preproccess_traffic(df)

In [21]:
df.loc[df.id_orden == 188]

,id_orden,file,estado,moneda,fecha_servicio,fecha_out,pasajero,monto_a_pagar,proveedor,codigo_iata,fec_sal,fecha_de_pago_proveedor
100,188,LI0036,OK,D,2025-11-17 00:00:00,2025-11-17 00:00:00,PETER CHAN X2,868.01,PASSION,ZBR,2025-11-17T00:00:00.000,2025-10-08 00:00:00


In [27]:
df.fecha_servicio = pd.to_datetime(df["fecha_servicio"]).dt.date

In [28]:
df.loc[df.pasajero == "PETER CHAN X2"]

,id_orden,file,estado,moneda,fecha_servicio,fecha_out,pasajero,monto_a_pagar,proveedor,codigo_iata,fec_sal,fecha_de_pago_proveedor
100,188,LI0036,OK,D,2025-11-17,2025-11-17 00:00:00,PETER CHAN X2,868.01,PASSION,ZBR,2025-11-17T00:00:00.000,2025-10-08 00:00:00
144,189,LI0036,IN,D,2025-12-01,2025-12-12 00:00:00,PETER CHAN X2,22.00,HOTASI,BUE,2025-11-17T00:00:00.000,2025-10-18 00:00:00
162,191,LI0036,OK,D,2025-11-27,2025-11-27 00:00:00,PETER CHAN X2,81.00,TSA BUENOS AIRES,BUE,2025-11-17T00:00:00.000,2025-10-18 00:00:00
169,192,LI0036,OK,D,2025-11-27,2025-11-27 00:00:00,PETER CHAN X2,82.00,ANDREA&FER,BUE,2025-11-17T00:00:00.000,2025-10-18 00:00:00
175,193,LI0036,OK,D,2025-11-24,2025-11-27 00:00:00,PETER CHAN X2,3227.34,HOTEL DAS CATARATAS (BELMOND),IGU,2025-11-17T00:00:00.000,2025-10-20 00:00:00
184,194,LI0036,OK,D,2025-11-20,2025-11-21 00:00:00,PETER CHAN X2,225.00,RATEHAWK,EXT,2025-11-17T00:00:00.000,2025-10-18 00:00:00
196,195,LI0036,PC,D,2025-11-21,2025-11-24 00:00:00,PETER CHAN X2,1391.18,IBEROSTAR GRAND AMAZON CRUISE,MAO,2025-11-17T00:00:00.000,2025-10-18 00:00:00
197,196,LI0036,OK,D,2025-12-16,2025-12-17 00:00:00,PETER CHAN X2,120.00,HOTEL & HACIENDA LA CARAVEDO,ICP,2025-11-17T00:00:00.000,2025-10-18 00:00:00
